In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from random import sample

In [29]:
#initialize parameters
gamma=10**(-8)
lbd=10**8
M=10
h=10**(-4)
c=0.05
m0=0

In [4]:
#function to perform stochastic gradient langevin dynamic in 1d
def sgld(X,M,h,gamma,lbd,n_sim):
    ms=np.zeros(M)
    ms[0]=m0
    dB = np.random.randn(M)   
    for i in range(1,M):
        dL=1-1/c*(np.sum(X>=ms[i-1]))/n_sim
        ms[i]= ms[i-1]-(dL+gamma*ms[i-1])*h + np.sqrt(2*lbd**(-1)*h)*dB[i]
    return ms

In [23]:
#This cell computes the AVaR in the 1-dimensional case
n_sim=10000
X=np.random.randn(1,n_sim)
#compute value at risk
var=sgld(X,M,h,gamma,lbd,n_sim)
#compute average value at risk
values=np.zeros((n_sim,M))
for i in range(n_sim):
    values[i]=np.amax(((X[0][i]-var),np.zeros(M)),axis=0)
L1=np.mean(values/0.05,axis=0)+var+gamma/2*var**2

In [37]:
#function to compute value at risk in the d-dimensional case
#S is the portfolio
def var_d(S,M):
    d=np.shape(S)[1]
    e1=np.zeros(d+1)
    dB = np.sqrt(h) * np.random.randn(M)   
    zs=np.zeros((M,d+1))
    zs[0][0]=10
    #preform stochastic gradient langevin descent
    for i in range(1,M):
        g=np.exp(zs[i-1][1:])/sum(np.exp(zs[i-1][1:]))
        f=np.dot(g,S.T)
        e1[0]=1+gamma*zs[i-1][0]
        partialg=np.zeros((d,d))
        for k in range(0,d):
            partialg[k]=(-np.exp(np.fmin(zs[i-1][k], 0)) / (1 + np.exp(-np.abs(zs[i-1][k]))))*(np.exp(np.fmin(zs[i-1][1:], 0)) / (1 + np.exp(-np.abs(zs[i-1][1:]))))/(np.sum(np.exp(np.fmin(zs[i-1][1:],0)) / (1 + np.exp(-np.abs(zs[i-1][1:])))))**2
        for j in range(0,d):
            partialg[j][j]=np.exp(zs[i-1][j])*(np.sum(np.exp(zs[i-1]))-np.exp(zs[i-1][j]))/(np.sum(np.exp(zs[i-1][1:])))**2
        g_hat=np.matmul(partialg,S.T)
        df=np.append(-np.ones((1,n_sim)),g_hat,axis=0)
        dL=e1+1/c*np.sum(df*(f>=zs[i-1][0]),axis=1)/n_sim
        zs[i]= zs[i-1]-(dL)*h + np.sqrt(2*lbd**(-1))*dB[i]
    return zs

In [38]:
n_sim=10000
#Here S is a portfolio of two Gaussian random variables
S1=np.random.normal(1, 4, size=((n_sim,1)))
S2=np.random.normal(0, 1, size=((n_sim,1)))
S=np.concatenate((S1,S2),axis=1)
#compute value at risk
zs=var_d(S,M)
#compute conditional value at risk
ms=zs[:,0]
values=np.zeros((n_sim,M))
g=np.exp(zs[-1][1:])/sum(np.exp(zs[-1][1:]))
f=np.dot(g,S.T)
for i in range(n_sim):
    values[i]=np.amax(((f[i]-ms),np.zeros(M)),axis=0)
avar=np.mean(values/c,axis=0)+ms+gamma/2*ms**2